# Exploratory Data Analysis(EDA) 

In [1]:
# import libraries and module
import pandas as pd
import numpy as np

In [4]:

import sys
sys.path.append("/content/drive/MyDrive/UnfathomableGuy/DataScience/Projects/Breast_Cancer_Prediction/Model_Building_Steps/utility_modules")
from data_wrangling import *
df.head()


In [3]:
import urllib.request

In [2]:
url = "https://raw.githubusercontent.com/Gkchandora/Breast_Cancer_Prediction/main/Model_Building_Steps/"

In [5]:
a = urllib.request.urlopen(url)

In [7]:
eval(a.read())

SyntaxError: ignored

In [4]:
import httpimport

In [5]:
with httpimport.remote_repo(['utility_modules'], url):
    import utility_modules

In [8]:
dir(utility_modules)

['__all__',
 '__builtins__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']